## 5.2 Celery

Celery is another queue management system. In contrast RQ, it is broker agnostric and can use various software as broker, such as Redis, RbiitMQ or Amazon SQS. Moreover if you are brave enough, it is possible to write your own driver.

Celery also needs a backend for storing the results of the job. It supports a bunch of solution, such as Redis, MongoDB, SQL databases, ElasticSearch, files, etc. Just like for brokers, you can also write your own.

Celery implements its own serialization format for its jobs. However, this format is not specifid to Python. That means it is possible to implement job creators or consumers in different languages; there are already clients in PHP and Javascript.

In Celery, tasks are functions that can be called asynchronously. When called, Celery puts them in the broker queue for execution. Remote workers then execute the tasks, putting the task results into the backend.
When called, a task returns a celery.result.AsyncResult object. This object offers a principal method, get, which returns that result as soon as it is available, blocking the program in the meantime..

In [ ]:
import celery

app = celery.Celery('celery-task', 
                   broker='redis://localhost',
                   backend='redis://localhost')

@app.task
def add(x, y):
    return x + y


if __name__ == '__main__':
    result =add.delay(4, 4)
    print("Task state: %s" % result.state)
    print("Result: %s" % result.get())
    print("Task state: %s" % result.state)

celery task. The celery application is created with the main module name as its first argument, and the th URL to access the broker and backends. 

The app.task function decorator registers the add task so it can be used asynchronously in the application, leveragin Celery workers for execution.

Once run, this program prints the following:

The celery command line tool provides a broad set of commands to manipulate and inspect the jobs queue and the workers. It is in charge of starting the actual workers:

### 5.2.1 Handling Failures

Task executions might fail, and in this case it is crucial to handle that properly. It is common for tasks to depend on external services, such as a remote database or a REST API. Connection failure might be transient; it is therefore better to deal with defeat and retry later.